### Local data (HDF5 Example)
This notebook walks through using Neuroglancer to view and interact with local data sources.

In [1]:
import neuroglancer
import h5py
import requests
import numpy as np
import itertools
from tqdm import tqdm
from IPython.display import display, HTML

In [2]:
# Download one of the CREMI datasets (https://cremi.org/)
url = "https://cremi.org/static/data/sample_A_20160501.hdf"  # 175M
r = requests.get(url, stream=True)
with open("sampleA.hdf", "wb") as f:
    for data in tqdm(r.iter_content(chunk_size=2**20), unit='MB', mininterval=1, total=int(r.headers.get('content-length', 0))//2**20):
        f.write(data)

168MB [01:52,  1.49MB/s]                         


In [3]:
import os
from urllib.parse import urlparse
import neuroglancer
from IPython.display import display, HTML

# Uncomment the following if you intend to run from a remote server:
# neuroglancer.set_server_bind_address('0.0.0.0')

neuroglancer.set_static_content_source(url="https://neuroglancer-demo.appspot.com/python")
viewer = neuroglancer.Viewer()

if "BINDER_URL" in os.environ: 
    viewer_url = "https://{0}{1}proxy/{2}{3}".format(
        "hub.gke.mybinder.org",
        os.environ["JUPYTERHUB_SERVICE_PREFIX"],
        urlparse(viewer.get_viewer_url()).port,
        urlparse(viewer.get_viewer_url()).path)
else:
    viewer_url = viewer.get_viewer_url()

display(HTML('Neuroglancer link: <a href="{0}">{0}</a>'.format(viewer_url)))

In [4]:
# Open HDF file and create local layers for image data and segmentation
h5f = h5py.File("sampleA.hdf", 'r')
with viewer.txn() as s:
    # Raw data
    s.layers['raw'] = neuroglancer.ImageLayer(
        source = neuroglancer.LocalVolume(data=h5f['volumes/raw'], voxel_size=(4, 4, 40))
    )
    # Segmentation data
    s.layers['neuron_ids'] = neuroglancer.SegmentationLayer(
        source = neuroglancer.LocalVolume(data=h5f['volumes/labels/neuron_ids'], voxel_size=(4, 4, 40))
    )

In [5]:
# Create and populate layers for pre-, post- and connectors

# Unique ID for neuroglancer objects
ngid = itertools.count(start=1)

(pre_sites, post_sites, connectors) = ([], [], [])
locations = []
#for location in h5f['annotations/locations']:
#    locations.append(np.flip(location))
# Result: a map from {id} -> {x,y,z}
locations = dict(zip(h5f['annotations/ids'], h5f['annotations/locations']))
    
for (pre, post) in h5f['annotations/presynaptic_site/partners']:
    pre_site = np.flip(locations[pre])
    post_site = np.flip(locations[post])
    
    pre_sites.append(neuroglancer.EllipsoidAnnotation(center=pre_site, radii=(40,40,40), id=next(ngid)))
    post_sites.append(neuroglancer.EllipsoidAnnotation(center=post_site, radii=(40,40,40), id=next(ngid)))
    connectors.append(neuroglancer.LineAnnotation(point_a=pre_site,point_b=post_site, id=next(ngid)))

with viewer.txn() as s:
    s.layers['connetors'] = neuroglancer.AnnotationLayer(
        voxel_size=(1,1,1),
        filter_by_segmentation=False,
        annotation_color='#ffff00',
        annotations=connectors,
    )
    s.layers['pre_sites'] = neuroglancer.AnnotationLayer(
        voxel_size=(1,1,1),
        filter_by_segmentation=False,
        annotation_color='#00ff00',
        annotations=pre_sites,
    )
    s.layers['post_sites'] = neuroglancer.AnnotationLayer(
        voxel_size=(1,1,1),
        filter_by_segmentation=False,
        annotation_color='#ff00ff',
        annotations=post_sites,
    )